In [1]:
######################################
import warnings
warnings.filterwarnings('ignore')
import json
import csv
import requests
import os.path
import oauth2client
import gspread
import json
from   oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup, NavigableString

In [2]:
from api_yelo import get_all_merchants
from api_yelo import json_to_string
#from api_yelo import insert_product
from api_yelo import get_catalog
from api_yelo import update_product
from api_yelo import disable_product
from api_yelo import delete_product
from api_yelo import get_all_products
from api_yelo import get_product
from api_yelo import get_category_name
from api_yelo import concat_images
from api_yelo import chang1

In [3]:
#######################################################
# Step 0: Share Google Drive Worksheet to this email ..
#######################################################
#'client_email': 'catalogodtt@iconic-post-192814.iam.gserviceaccount.com'

In [4]:
##############################################
# Step 1: Read Google Drive Excel Worksheet ..
##############################################
json_key = json.load(open('creds.json'))
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(creds)
sheet_victor_antiveros = client.open('Codificacion Productos Kiwua API').worksheet('Víctor Antivero (P&B)')

In [5]:
api_key             = 'aafb702727d770509efb0b2e7576ce4f'
marketplace_user_id = 157755
user_id             = int(get_all_merchants().get("Víctor Antivero"))

In [6]:
merchants=get_all_merchants()
merchants

{'Víctor Antivero': 318980,
 'Tony como Proveedor': 290144,
 'Tienda': 291239,
 'test': 319905,
 'Subway': 314801,
 'Quick Meats': 332562,
 'Portonovo': 308994,
 'PIAZZA Automercados': 321621,
 'Monte Cacao': 342619,
 'Kit para Kiwuanos': 362595,
 'Johana Silva': 313384,
 'INCA': 308071,
 'Haidee Rodriguez': 313481,
 'Fundación Nativo': 267942,
 'El Huerto de Altamira': 319885,
 'Distribuidora TT': 157756,
 'Distribuidora Tienda Tienda': 321750,
 'Disjobel': 157755,
 "Carnicería ZOE's": 157758,
 'Barco Pesca': 157757}

In [7]:
# delete_product(product_id=3894030,user_id=user_id)
# delete_product(product_id=3894039,user_id=user_id)
# delete_product(product_id=3894080,user_id=user_id)

In [8]:
merchant_to_set_markup = dict()
merchant_to_set_markup[merchants['PIAZZA Automercados']]   = 1.08
merchant_to_set_markup[merchants['Distribuidora TT']]      = 1.08
merchant_to_set_markup[merchants['El Huerto de Altamira']] = 1.08
merchant_to_set_markup[merchants['Víctor Antivero']]       = 1.15
merchant_to_set_markup[merchants['Disjobel']]              = 1.15
merchant_to_set_markup

{321621: 1.08, 157756: 1.08, 319885: 1.08, 318980: 1.15, 157755: 1.15}

In [9]:
user_bs_dollar = {
 'Víctor Antivero': 318980}

In [10]:
url_bcv='http://www.bcv.org.ve/estadisticas/tipo-de-cambio'

In [11]:
def concat_images(*args):
    if len(args)==1:
        return list(args)
    else:
        return [i for i in args]

def verify_insert_conditions(category_id,price,description,image_url):
    if(category_id!=None) and (price>0.1) and (image_url!=["", "", "", "", "", ""]):
        resp = True
    else:
        resp = False
    return resp

def update_product(user_id,product_id,name,price,image_url=None,description=None):
    body=dict()
    body["api_key"]             = "aafb702727d770509efb0b2e7576ce4f"
    body["marketplace_user_id"] = 157755
    body["user_id"]             = user_id
    body["product_id"]          = product_id
    body["name"]                = name
    body["price"]               = price
    if description!=None: 
        body["description"]     = description
    if image_url!=None:
        body["multi_image_url"] = [image_url]

    headers = {
             "Content-Type"      : "application/json",
             "cache-control"     : "no-cache"
             }
    
    #print('update\n',json_to_string(body))

    try:
        url = "https://api.yelo.red/open/product/edit"
        response = requests.post(url     = url,
                                 data    = json_to_string(body),
                                 headers = headers)
        print('update:',response.json())
        print('-'*80)
        return 0
    except:
        return 1
    
def insert_product(user_id, name, price, category_id,image_url, description=None, long_description=None):
    payload = dict()
    payload["api_key"]               = "aafb702727d770509efb0b2e7576ce4f"
    payload["user_id"]               = user_id
    payload["marketplace_user_id"]   = 157755
    payload["name"]                  = name
    payload["name_json"]             = {"es": name}
    if description!=None:
        payload["description"]           = description
        payload["description_json"]      = {"es": description}
    if long_description!=None:
        payload["long_description_json"] = {"es": long_description}
    payload["parent_category_id"]    = category_id
    if image_url!=None:
        payload["multi_image_url"]       = image_url
    payload["inventory_enabled"]     = 0
    payload["available_quantity"]    = 100
    payload["price"]                 = price
    body     = json_to_string(payload)
    print('insert\n',body.replace('\'','\"'))
    headers  = {'Content-Type': "application/json",'cache-control': "no-cache"}
    try:
        response = requests.request("POST",
                                    "https://api.yelo.red/open/product/add", 
                                    data=body, 
                                    headers=headers)
        print('insert:',name,response.json())
        print('\n'+'-'*80)
        return 0
    except:
        return 1

In [12]:
html      = requests.get(url_bcv).text
soup      = BeautifulSoup(html)

tasa=soup.find('div',{'id':'dolar'})
tasa_cambio_bs_dollar = float(tasa.strong.text.replace(' ','').replace('.','').replace(',','.'))
tasa_cambio_bs_dollar

171072.85

In [13]:
categorias_victor_antiveros = get_catalog(user_id=user_id)
categorias_victor_antiveros

{'Cuidado Personal': 726553,
 'Jabón & Gel': 1098968,
 'Desodorante': 1098969,
 'Alimento': 1098971}

In [14]:
products_kiwua = get_all_products(user_id)

In [15]:
###############################################
# Step 2. Update products in kiwua...
###############################################
products = sheet_victor_antiveros.get_all_records()
factor   = merchant_to_set_markup[merchants['Víctor Antivero']]
cont1=0
cont2=0
for product in products:
        name             = product['Nombre Kiwua']
        category_id      = categorias_victor_antiveros.get(product['Categoría Kiwua'])
        price            = float(product['Precio (Convenio 01)'].replace('.','').replace(',','.'))
        price            = price/tasa_cambio_bs_dollar
        price            = round(price*factor,2)
        description      = product['Descripción Kiwua (Presentación)']
        long_description = product['Descripción Larga Kiwua (Beneficio & usos)']
        image_url        = concat_images(product['URL Imagen 1'],
                                         product['URL Imagen 2'],
                                         product['URL Imagen 3'],
                                         product['URL Imagen 4'],
                                         product['URL Imagen 5'],
                                         product['URL Imagen 6'])
        resp1 = 1
        resp2 = 1
        try:
            product_id  = products_kiwua[chang1(name)][0]
            disable_product(user_id,[product_id],1)
            resp1       = update_product(  user_id      = user_id
                                           ,product_id  = product_id
                                           ,name        = name
                                           ,price       = price
                                           ,image_url   = image_url
                                           ,description = description)
            if resp1==0:
                cont1+=1
        except KeyError:
            if verify_insert_conditions(category_id,price,description,image_url):
                resp2 = insert_product(user_id          = user_id, 
                                       category_id      = category_id,
                                       name             = name,
                                       price            = price,
                                       description      = description,
                                       long_description = long_description,
                                       image_url        = image_url)
            else:
                print('Register imcomplete',[category_id,price,description,image_url])
            if resp2==0:
                cont2+=1
print('Updated :',cont1,' records')
print('Inserted:',cont2,' records')

Register imcomplete [None, 1.74, '', ['', '', '', '', '', '']]
Register imcomplete [None, 1.74, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.0, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.0, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.0, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.0, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.05, '', ['', '', '', '', '', '']]
Register imcomplete [None, 0.05, '', ['', '', '', '', '', '']]
Register imcomplete [726553, 0.06, 'Dencofen Gel 20', ['https://res.cloudinary.com/kiwua/image/upload/v1587528069/Victor%20Antivero/Definitivas/Dencofen_1_16_C-Gel_20X48_orevj6.jpg', '', '', '', '', '']]
insert
 {"api_key":"aafb702727d770509efb0b2e7576ce4f","user_id":318980,"marketplace_user_id":157755,"name":"Dencofen Gel 40gr","name_json":{"es":"Dencofen Gel 40gr"},"description":"Dencofen Gel 40gr","description_json":{"es":"Dencofen Gel 40gr"},"long_description_json":{"es":"De efectividad comp

insert: Dencorub Ice 80g {'message': 'Exitoso', 'status': 200, 'data': {'id': 5887187, 'multi_image_url': ['https://res.cloudinary.com/kiwua/image/upload/v1587528080/Victor%20Antivero/Definitivas/Dencorub_Ice_80X24_wbpihg.jpg', '', '', '', '', ''], 'thumb_url': 'https://yelodotred.s3.amazonaws.com/yelo_products/thumb-250-250-Dencorub_Ice_80X24_wbpihg.jpg'}}

--------------------------------------------------------------------------------
insert
 {"api_key":"aafb702727d770509efb0b2e7576ce4f","user_id":318980,"marketplace_user_id":157755,"name":"Dencorub Ice 40g","name_json":{"es":"Dencorub Ice 40g"},"description":"Dencorub Ice en cajas de 24 unidades 40g","description_json":{"es":"Dencorub Ice en cajas de 24 unidades 40g"},"long_description_json":{"es":"Una línea de productos analgésicos de uso tópico para el alivio del dolor de músculos y articulaciones ocasionados por el ejercicio físico"},"parent_category_id":726553,"multi_image_url":["https://res.cloudinary.com/kiwua/image/upload/v1

insert: Gel Antibacterial 220 {'message': 'Exitoso', 'status': 200, 'data': {'id': 5887192, 'multi_image_url': ['https://res.cloudinary.com/kiwua/image/upload/v1587528084/Victor%20Antivero/Definitivas/Diox_Gel_Antibacterial_220_fhusfe.jpg', '', '', '', '', ''], 'thumb_url': 'https://yelodotred.s3.amazonaws.com/yelo_products/thumb-250-250-Diox_Gel_Antibacterial_220_fhusfe.jpg'}}

--------------------------------------------------------------------------------
Register imcomplete [1098968, 0.45, '', ['', '', '', '', '', '']]
Register imcomplete [1098968, 0.47, '', ['', '', '', '', '', '']]
insert
 {"api_key":"aafb702727d770509efb0b2e7576ce4f","user_id":318980,"marketplace_user_id":157755,"name":"","name_json":{"es":""},"description":"","description_json":{"es":""},"long_description_json":{"es":"Es Antiséptico y está especialmente formulado para la limpieza, desinfección y humectación de la piel"},"parent_category_id":726553,"multi_image_url":["https://res.cloudinary.com/kiwua/image/uploa

insert: Dioxogen Solución 115 cm3 {'message': 'Exitoso', 'status': 200, 'data': {'id': 5887198, 'multi_image_url': ['https://res.cloudinary.com/kiwua/image/upload/v1587528092/Victor%20Antivero/Definitivas/Dioxogen_Soluci%C3%B3n_115_pssrkh.jpg', '', '', '', '', ''], 'thumb_url': 'https://yelodotred.s3.amazonaws.com/yelo_products/thumb-250-250-Dioxogen_Soluci%25C3%25B3n_115_pssrkh.jpg'}}

--------------------------------------------------------------------------------
insert
 {"api_key":"aafb702727d770509efb0b2e7576ce4f","user_id":318980,"marketplace_user_id":157755,"name":"Dioxogen Solución 230 cm3","name_json":{"es":"Dioxogen Solución 230 cm3"},"description":"Dioxogen Solución Antiséptica en cajas de 24 unidades 230 cm3","description_json":{"es":"Dioxogen Solución Antiséptica en cajas de 24 unidades 230 cm3"},"long_description_json":{"es":"Excelente para eliminar todo tipo de microorganismos, tales como bacterias, virus, hongos y esporas. Es recomendado su uso para desinfectar cualquie

insert: Wampole Emulsión Fresa 360 ml {'message': 'Exitoso', 'status': 200, 'data': {'id': 5887206, 'multi_image_url': ['https://res.cloudinary.com/kiwua/image/upload/v1587528101/Victor%20Antivero/Definitivas/Wampole_Emuls_Fresa_360X24_thh7os.jpg', '', '', '', '', ''], 'thumb_url': 'https://yelodotred.s3.amazonaws.com/yelo_products/thumb-250-250-Wampole_Emuls_Fresa_360X24_thh7os.jpg'}}

--------------------------------------------------------------------------------
Register imcomplete [1098971, 1.07, '', ['', '', '', '', '', '']]
insert
 {"api_key":"aafb702727d770509efb0b2e7576ce4f","user_id":318980,"marketplace_user_id":157755,"name":"Wampole Emulsión Naranja 240 ml","name_json":{"es":"Wampole Emulsión Naranja 240 ml"},"description":"Wampole Emulsión Naranja caja de 12 unidades 240 ml.","description_json":{"es":"Wampole Emulsión Naranja caja de 12 unidades 240 ml."},"long_description_json":{"es":"Wampole es una línea de productos de complementos vitamínicos hechos para los niños, pro

In [ ]:
products=sheet_victor_antiveros.get_all_records()

In [ ]:
float(products[0]['Precio (Convenio 01)'].replace('.','').replace(',','.'))

In [ ]:
products = sheet.get_all_records()

all_cells = sheet.range('A1:A3')

print(all_cells[0].value)

B2 = sheet.acell('B2').value

print(B2)

coord = sheet.cell(2, 1).value

print(coord)

row = sheet.row_values(1) # first row

col = sheet.col_values(2) # models

sheet.update_acell('C2', 'Blue')

sheet.update_cell(2, 1, 6666)

coord = sheet.cell(2, 1).value

print(coord)